In [ ]:
# Setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import sys

repo_root = Path().resolve().parents[2]
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

from modules._import_helper import safe_import_from
from sklearn.metrics import (
    roc_auc_score, average_precision_score, accuracy_score,
    precision_score, recall_score, f1_score, confusion_matrix,
    roc_curve, precision_recall_curve
)
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

set_seed = safe_import_from('00_repo_standards.src.mlphys_core.seeding', 'set_seed')
load_data, get_feature_columns, split_data = safe_import_from(
    '03_ml_tabular_foundations.src.data',
    'load_data', 'get_feature_columns', 'split_data'
)

set_seed(42)
reports_dir = Path("../reports")
reports_dir.mkdir(exist_ok=True)

print("✅ Setup complete")

## 1. Why Baselines Matter

**Baseline**: A simple model that provides a reference point.

**Purpose**:
- Establish minimum acceptable performance
- Quantify value of complex models
- Sanity check: complex model must beat baseline
- Debug: if baseline fails, data may be broken

**Types of baselines**:
1. **Constant predictor**: Predict majority class (classification) or mean (regression)
2. **Random predictor**: Random guessing
3. **Linear model**: Logistic/linear regression (fast, interpretable)
4. **Domain heuristic**: Physics-based rules (if available)

In [ ]:
# Load and split data
df = load_data()
feature_cols = get_feature_columns(df)
X = df[feature_cols].values
y = df['is_signal'].values

X_train, X_val, X_test, y_train, y_val, y_test = split_data(
    X, y, test_size=0.2, val_size=0.2, random_state=42
)

print(f"Dataset: {X.shape[0]:,} samples, {X.shape[1]} features")
print(f"Class balance: {y.mean():.1%} signal, {1-y.mean():.1%} background")
print(f"\nSplit sizes: Train={len(y_train):,}, Val={len(y_val):,}, Test={len(y_test):,}")

## 2. Baseline 1: Majority Class Predictor

**Strategy**: Always predict the most common class.

**When useful**: Establishes floor performance for imbalanced datasets.

In [ ]:
# Majority class baseline
majority_baseline = DummyClassifier(strategy='most_frequent')
majority_baseline.fit(X_train, y_train)

# Predictions (probabilities are always [0.9, 0.1] for background-heavy dataset)
y_pred_train_maj = majority_baseline.predict(X_train)
y_pred_val_maj = majority_baseline.predict(X_val)
y_pred_test_maj = majority_baseline.predict(X_test)

# Metrics
acc_train_maj = accuracy_score(y_train, y_pred_train_maj)
acc_val_maj = accuracy_score(y_val, y_pred_val_maj)
acc_test_maj = accuracy_score(y_test, y_pred_test_maj)

print("Baseline 1: Majority Class Predictor")
print("=" * 60)
print(f"Strategy: Always predict class {majority_baseline.classes_[majority_baseline.class_prior_.argmax()]} (background)")
print(f"\nAccuracy:")
print(f"  Train: {acc_train_maj:.1%}")
print(f"  Val:   {acc_val_maj:.1%}")
print(f"  Test:  {acc_test_maj:.1%}")
print(f"\n⚠️ Lesson: {acc_test_maj:.1%} accuracy by just guessing majority class!")
print("   → Accuracy is MISLEADING for imbalanced data")
print("   → Need better metrics: AUC-ROC, PR-AUC")

## 3. Baseline 2: Logistic Regression

**Strategy**: Simple linear model (fast to train, interpretable).

**Why it's a good baseline**:
- Captures linear relationships
- Provides probability estimates
- Fast to train and predict
- Interpretable coefficients

In [ ]:
# Logistic regression baseline (with proper pipeline)
logistic_baseline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(max_iter=1000, random_state=42))
])

logistic_baseline.fit(X_train, y_train)

# Predictions (probabilities)
y_pred_train_lr = logistic_baseline.predict_proba(X_train)[:, 1]
y_pred_val_lr = logistic_baseline.predict_proba(X_val)[:, 1]
y_pred_test_lr = logistic_baseline.predict_proba(X_test)[:, 1]

# Compute metrics
def compute_classification_metrics(y_true, y_pred_proba, threshold=0.5):
    """Compute comprehensive classification metrics."""
    y_pred = (y_pred_proba >= threshold).astype(int)
    
    return {
        'Accuracy': accuracy_score(y_true, y_pred),
        'Precision': precision_score(y_true, y_pred, zero_division=0),
        'Recall': recall_score(y_true, y_pred, zero_division=0),
        'F1': f1_score(y_true, y_pred, zero_division=0),
        'AUC-ROC': roc_auc_score(y_true, y_pred_proba),
        'PR-AUC': average_precision_score(y_true, y_pred_proba)
    }

metrics_train_lr = compute_classification_metrics(y_train, y_pred_train_lr)
metrics_val_lr = compute_classification_metrics(y_val, y_pred_val_lr)
metrics_test_lr = compute_classification_metrics(y_test, y_pred_test_lr)

print("\nBaseline 2: Logistic Regression")
print("=" * 60)
print(f"{'Metric':<12} {'Train':<10} {'Val':<10} {'Test':<10}")
print("-" * 60)
for metric in metrics_train_lr.keys():
    print(f"{metric:<12} {metrics_train_lr[metric]:<10.4f} "
          f"{metrics_val_lr[metric]:<10.4f} {metrics_test_lr[metric]:<10.4f}")

print(f"\n🎯 Key Observations:")
print(f"  • AUC-ROC = {metrics_test_lr['AUC-ROC']:.3f}: Model discriminates signal from background")
print(f"  • PR-AUC = {metrics_test_lr['PR-AUC']:.3f}: Performance on minority class (signal)")
print(f"  • Accuracy = {metrics_test_lr['Accuracy']:.3f}: Inflated by majority class")

## 4. Understanding Metrics for Imbalanced Data

**Problem**: With 10% signal rate, predicting all background gives 90% accuracy!

**Solution**: Use metrics that focus on discrimination ability.

### 4.1 ROC-AUC: Discrimination Ability

**ROC Curve**: True Positive Rate vs. False Positive Rate at all thresholds.

**AUC-ROC**:
- Probability that model ranks random positive higher than random negative
- Range: [0, 1], random = 0.5, perfect = 1.0
- **Insensitive to class imbalance**

**When to use**: Class balance doesn't strongly affect business goal.

In [ ]:
# Plot ROC curve
fpr, tpr, thresholds_roc = roc_curve(y_test, y_pred_test_lr)

fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(fpr, tpr, linewidth=2.5, label=f'Logistic (AUC={metrics_test_lr["AUC-ROC"]:.3f})', 
        color='steelblue')
ax.plot([0, 1], [0, 1], 'k--', linewidth=1.5, label='Random (AUC=0.500)', alpha=0.6)
ax.set_xlabel('False Positive Rate', fontsize=12)
ax.set_ylabel('True Positive Rate', fontsize=12)
ax.set_title('ROC Curve: Logistic Regression Baseline', fontsize=13, fontweight='bold')
ax.legend(fontsize=11, loc='lower right')
ax.grid(alpha=0.3)
ax.set_xlim([-0.02, 1.02])
ax.set_ylim([-0.02, 1.02])

plt.tight_layout()
plt.savefig(reports_dir / '03_roc_curve_baseline.png', dpi=150, bbox_inches='tight')
plt.show()

print("📊 ROC-AUC Interpretation:")
print(f"  AUC = {metrics_test_lr['AUC-ROC']:.3f}")
print(f"  → {metrics_test_lr['AUC-ROC']*100:.1f}% chance model ranks signal higher than background")
print(f"  → Far better than random (0.500)")

### 4.2 PR-AUC: Performance on Minority Class

**Precision-Recall Curve**: Precision vs. Recall at all thresholds.

**PR-AUC**:
- Average precision across recall levels
- **Sensitive to class imbalance** (good for minority class focus)
- Baseline = prevalence (10% in our case)

**When to use**: Minority class is primary concern (fraud, disease, signal).

In [ ]:
# Plot Precision-Recall curve
precision, recall, thresholds_pr = precision_recall_curve(y_test, y_pred_test_lr)

fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(recall, precision, linewidth=2.5, 
        label=f'Logistic (PR-AUC={metrics_test_lr["PR-AUC"]:.3f})', 
        color='coral')
ax.axhline(y=y_test.mean(), color='k', linestyle='--', linewidth=1.5, 
           label=f'Random (PR-AUC={y_test.mean():.3f})', alpha=0.6)
ax.set_xlabel('Recall', fontsize=12)
ax.set_ylabel('Precision', fontsize=12)
ax.set_title('Precision-Recall Curve: Logistic Regression Baseline', 
             fontsize=13, fontweight='bold')
ax.legend(fontsize=11, loc='upper right')
ax.grid(alpha=0.3)
ax.set_xlim([-0.02, 1.02])
ax.set_ylim([-0.02, 1.02])

plt.tight_layout()
plt.savefig(reports_dir / '03_pr_curve_baseline.png', dpi=150, bbox_inches='tight')
plt.show()

print("📊 PR-AUC Interpretation:")
print(f"  PR-AUC = {metrics_test_lr['PR-AUC']:.3f}")
print(f"  Baseline (random) = {y_test.mean():.3f} (prevalence)")
print(f"  → {(metrics_test_lr['PR-AUC'] - y_test.mean()) / y_test.mean() * 100:.0f}% improvement over random")

## 5. Confusion Matrix: Understanding Errors

**Confusion Matrix**: Breakdown of predictions into TP, TN, FP, FN.

**Business interpretation**:
- **False Positive (FP)**: Background misclassified as signal → wasted resources
- **False Negative (FN)**: Signal misclassified as background → missed discovery

In [ ]:
# Confusion matrix at default threshold (0.5)
y_pred_test_lr_binary = (y_pred_test_lr >= 0.5).astype(int)
cm = confusion_matrix(y_test, y_pred_test_lr_binary)

fig, ax = plt.subplots(figsize=(7, 6))
im = ax.imshow(cm, cmap='Blues', alpha=0.8)

# Add text annotations
for i in range(2):
    for j in range(2):
        text = ax.text(j, i, f'{cm[i, j]:,}\n({cm[i, j]/cm.sum()*100:.1f}%)',
                      ha='center', va='center', fontsize=13, fontweight='bold',
                      color='white' if cm[i, j] > cm.max()/2 else 'black')

ax.set_xticks([0, 1])
ax.set_yticks([0, 1])
ax.set_xticklabels(['Pred: Background', 'Pred: Signal'], fontsize=11)
ax.set_yticklabels(['True: Background', 'True: Signal'], fontsize=11)
ax.set_title('Confusion Matrix (Threshold=0.5)', fontsize=13, fontweight='bold')
ax.set_xlabel('Predicted Label', fontsize=12)
ax.set_ylabel('True Label', fontsize=12)

# Colorbar
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Count', fontsize=11)

plt.tight_layout()
plt.savefig(reports_dir / '03_confusion_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

# Compute error rates
tn, fp, fn, tp = cm.ravel()
fpr_val = fp / (fp + tn)
fnr_val = fn / (fn + tp)

print("📊 Confusion Matrix Analysis:")
print(f"  True Negatives (TN):  {tn:,} (correct background)")
print(f"  False Positives (FP): {fp:,} (background → signal)")
print(f"  False Negatives (FN): {fn:,} (signal → background)")
print(f"  True Positives (TP):  {tp:,} (correct signal)")
print(f"\nError Rates:")
print(f"  False Positive Rate: {fpr_val:.1%} (FP / all background)")
print(f"  False Negative Rate: {fnr_val:.1%} (FN / all signal)")

## 6. Threshold Selection: Aligning Metrics with Decisions

**Problem**: Default threshold (0.5) may not be optimal for business goal.

**Example**: In particle physics, prefer high recall (catch all signals) even if precision suffers.

**Solution**: Select threshold based on cost/benefit tradeoff.

### 6.1 Scenario: Maximize Recall at 95% Precision

**Goal**: Catch as many signal events as possible while maintaining 95% precision.

**Why**: We can afford some background (cheap to filter later), but missing signal is expensive.

In [ ]:
# Find optimal threshold for target precision
target_precision = 0.95

# Compute precision/recall at all thresholds
precision_all, recall_all, thresholds_all = precision_recall_curve(y_test, y_pred_test_lr)

# Find threshold achieving target precision
valid_idx = np.where(precision_all >= target_precision)[0]
if len(valid_idx) > 0:
    # Select threshold with highest recall among valid
    best_idx = valid_idx[np.argmax(recall_all[valid_idx])]
    optimal_threshold = thresholds_all[best_idx] if best_idx < len(thresholds_all) else 1.0
    optimal_precision = precision_all[best_idx]
    optimal_recall = recall_all[best_idx]
else:
    optimal_threshold = 1.0
    optimal_precision = 1.0
    optimal_recall = 0.0

print(f"Threshold Selection for P ≥ {target_precision:.0%}:")
print("=" * 60)
print(f"  Optimal threshold: {optimal_threshold:.4f}")
print(f"  Achieved precision: {optimal_precision:.1%}")
print(f"  Achieved recall: {optimal_recall:.1%}")

# Compare with default threshold
y_pred_optimal = (y_pred_test_lr >= optimal_threshold).astype(int)
metrics_optimal = {
    'Precision': precision_score(y_test, y_pred_optimal),
    'Recall': recall_score(y_test, y_pred_optimal),
    'F1': f1_score(y_test, y_pred_optimal)
}

print(f"\nComparison:")
print(f"  Default (0.5):  P={metrics_test_lr['Precision']:.3f}, R={metrics_test_lr['Recall']:.3f}, F1={metrics_test_lr['F1']:.3f}")
print(f"  Optimal ({optimal_threshold:.3f}): P={metrics_optimal['Precision']:.3f}, R={metrics_optimal['Recall']:.3f}, F1={metrics_optimal['F1']:.3f}")
print(f"\n💡 Trade-off: Sacrificed {(metrics_test_lr['Recall'] - metrics_optimal['Recall'])*100:.1f} percentage points recall")
print(f"             to gain {(metrics_optimal['Precision'] - metrics_test_lr['Precision'])*100:.1f} percentage points precision")

## 7. Bootstrap Confidence Intervals

**Problem**: Single metric value doesn't capture uncertainty.

**Solution**: Bootstrap to estimate confidence interval.

**Method**:
1. Resample test set with replacement
2. Compute metric on each resample
3. Calculate 95% CI from distribution

In [ ]:
# Bootstrap confidence intervals for AUC-ROC
def bootstrap_metric(y_true, y_pred, metric_fn, n_bootstrap=1000, alpha=0.05):
    """Compute bootstrap confidence interval for a metric."""
    np.random.seed(42)
    
    scores = []
    n_samples = len(y_true)
    
    for _ in range(n_bootstrap):
        # Resample with replacement
        idx = np.random.randint(0, n_samples, n_samples)
        y_true_boot = y_true[idx]
        y_pred_boot = y_pred[idx]
        
        # Compute metric
        if len(np.unique(y_true_boot)) > 1:  # Need both classes
            score = metric_fn(y_true_boot, y_pred_boot)
            scores.append(score)
    
    # Compute confidence interval
    lower = np.percentile(scores, alpha/2 * 100)
    upper = np.percentile(scores, (1 - alpha/2) * 100)
    mean = np.mean(scores)
    
    return mean, lower, upper, scores

# Bootstrap AUC-ROC
auc_mean, auc_lower, auc_upper, auc_dist = bootstrap_metric(
    y_test, y_pred_test_lr, roc_auc_score, n_bootstrap=1000
)

# Bootstrap PR-AUC
pr_auc_mean, pr_auc_lower, pr_auc_upper, pr_auc_dist = bootstrap_metric(
    y_test, y_pred_test_lr, average_precision_score, n_bootstrap=1000
)

print("Bootstrap 95% Confidence Intervals (n=1000):")
print("=" * 60)
print(f"AUC-ROC:  {auc_mean:.4f} [{auc_lower:.4f}, {auc_upper:.4f}]")
print(f"PR-AUC:   {pr_auc_mean:.4f} [{pr_auc_lower:.4f}, {pr_auc_upper:.4f}]")

# Visualize bootstrap distributions
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.hist(auc_dist, bins=30, alpha=0.7, color='steelblue', edgecolor='black')
ax1.axvline(auc_mean, color='red', linewidth=2, label=f'Mean={auc_mean:.4f}')
ax1.axvline(auc_lower, color='orange', linewidth=2, linestyle='--', label=f'95% CI')
ax1.axvline(auc_upper, color='orange', linewidth=2, linestyle='--')
ax1.set_xlabel('AUC-ROC', fontsize=12)
ax1.set_ylabel('Frequency', fontsize=12)
ax1.set_title('Bootstrap Distribution: AUC-ROC', fontsize=13, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(alpha=0.3)

ax2.hist(pr_auc_dist, bins=30, alpha=0.7, color='coral', edgecolor='black')
ax2.axvline(pr_auc_mean, color='red', linewidth=2, label=f'Mean={pr_auc_mean:.4f}')
ax2.axvline(pr_auc_lower, color='orange', linewidth=2, linestyle='--', label=f'95% CI')
ax2.axvline(pr_auc_upper, color='orange', linewidth=2, linestyle='--')
ax2.set_xlabel('PR-AUC', fontsize=12)
ax2.set_ylabel('Frequency', fontsize=12)
ax2.set_title('Bootstrap Distribution: PR-AUC', fontsize=13, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.savefig(reports_dir / '03_bootstrap_distributions.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n💡 Interpretation:")
print(f"  • We are 95% confident true AUC-ROC is in [{auc_lower:.4f}, {auc_upper:.4f}]")
print(f"  • Width of CI: {auc_upper - auc_lower:.4f}")
print(f"  • Narrow CI → stable estimate (sufficient test data)")

## 8. Metrics Summary Table

Consolidate all metrics for easy comparison.

In [ ]:
# Create metrics summary
metrics_summary = pd.DataFrame({
    'Model': ['Majority Baseline', 'Logistic Regression'],
    'Accuracy': [acc_test_maj, metrics_test_lr['Accuracy']],
    'Precision': [0.0, metrics_test_lr['Precision']],
    'Recall': [0.0, metrics_test_lr['Recall']],
    'F1': [0.0, metrics_test_lr['F1']],
    'AUC-ROC': [0.5, metrics_test_lr['AUC-ROC']],
    'PR-AUC': [y_test.mean(), metrics_test_lr['PR-AUC']]
})

print("\n📊 Baseline Model Comparison (Test Set):")
print("=" * 60)
print(metrics_summary.to_string(index=False))

# Save to CSV
metrics_summary.to_csv(reports_dir / '03_baseline_metrics.csv', index=False)
print(f"\n✅ Metrics saved to: {reports_dir / '03_baseline_metrics.csv'}")

## 9. Exercises

**Exercise 1**: Choose metrics for a business scenario

You're building a medical diagnosis model for a rare disease (1% prevalence):
- **False Negative**: Patient has disease but test says no → missed treatment (HIGH COST)
- **False Positive**: Patient doesn't have disease but test says yes → unnecessary treatment (LOW COST)

Which metrics should you prioritize? What threshold strategy would you use?

In [ ]:
# Your answer here:
# 


**Exercise 2**: Compute optimal threshold for custom cost function

Given:
- Cost of False Negative = $1000 (missed signal discovery)
- Cost of False Positive = $10 (wasted detector readout)

Write code to find the threshold that minimizes total cost on the test set.

In [ ]:
# Your implementation here:
# 


**Exercise 3**: When does R² fail?

Consider a regression problem predicting house prices. You have two models:
- Model A: R² = 0.85, predictions always within 20% of true price
- Model B: R² = 0.90, but occasionally predicts negative prices

Which model would you deploy? Why? What additional metrics would you check?

In [ ]:
# Your answer here:
# 


---
## Solutions

**Solution 1**: Medical diagnosis metrics

**Prioritized metrics**:

1. **Primary: Recall (Sensitivity)** ✅
   - Goal: Catch all positive cases (minimize FN)
   - Willing to accept more FP to avoid missing disease
   
2. **Secondary: Precision (PPV)** 
   - Control false alarm rate
   - But lower priority than recall
   
3. **Threshold strategy**:
   - Set threshold to achieve **high recall (≥95%)**
   - Accept lower precision (e.g., 30-50%)
   - Follow-up with confirmatory tests for positives
   
4. **Why NOT accuracy or AUC-ROC**:
   - Accuracy: Misleading with 1% prevalence (99% by predicting all negative)
   - AUC-ROC: Doesn't capture asymmetric costs
   - **Use PR-AUC instead** (focuses on minority class)

**Example threshold selection**:
```python
# Find threshold giving 95% recall
target_recall = 0.95
precision_all, recall_all, thresholds = precision_recall_curve(y_test, y_pred)
idx = np.where(recall_all >= target_recall)[0]
optimal_threshold = thresholds[idx[-1]]  # Lowest threshold with ≥95% recall
```

**Clinical decision rule**:
- Threshold low (e.g., 0.1) → catch all cases
- Positive screening → confirmatory diagnostic test
- Better to over-test than miss disease

In [ ]:
# Solution 1 (demonstration)
print("Medical Diagnosis Strategy:")
print("=" * 60)
print("Scenario: Rare disease (1% prevalence)")
print("  Cost(FN) >> Cost(FP)")
print("")
print("Recommended Approach:")
print("  1. Primary metric: Recall (≥95%)")
print("  2. Secondary metric: Precision (acceptable if >30%)")
print("  3. Use PR-AUC for model selection")
print("  4. Set low threshold for high recall")
print("  5. Confirmatory test for all positives")
print("")
print("Why this works:")
print("  • Screening catches all cases (high recall)")
print("  • Confirmatory test filters false positives")
print("  • Acceptable workflow: screen 1000 → 100 positive → 10 confirm → 10 treat")

**Solution 2**: Optimal threshold for custom cost function

In [ ]:
# Solution 2
def find_optimal_threshold_by_cost(y_true, y_pred_proba, cost_fn, cost_fp):
    """Find threshold minimizing total cost."""
    # Try all unique thresholds
    thresholds = np.linspace(0, 1, 1000)
    
    costs = []
    for threshold in thresholds:
        y_pred = (y_pred_proba >= threshold).astype(int)
        cm = confusion_matrix(y_true, y_pred)
        tn, fp, fn, tp = cm.ravel()
        
        total_cost = fn * cost_fn + fp * cost_fp
        costs.append(total_cost)
    
    # Find threshold with minimum cost
    optimal_idx = np.argmin(costs)
    optimal_threshold = thresholds[optimal_idx]
    optimal_cost = costs[optimal_idx]
    
    return optimal_threshold, optimal_cost, costs

# Find optimal threshold
cost_fn = 1000  # Cost of False Negative
cost_fp = 10    # Cost of False Positive

optimal_thresh, optimal_cost, all_costs = find_optimal_threshold_by_cost(
    y_test, y_pred_test_lr, cost_fn, cost_fp
)

# Evaluate at optimal threshold
y_pred_cost_optimal = (y_pred_test_lr >= optimal_thresh).astype(int)
cm_optimal = confusion_matrix(y_test, y_pred_cost_optimal)
tn, fp, fn, tp = cm_optimal.ravel()

print("Optimal Threshold by Cost Minimization:")
print("=" * 60)
print(f"Cost parameters:")
print(f"  Cost(FN) = ${cost_fn:,}")
print(f"  Cost(FP) = ${cost_fp:,}")
print(f"  Cost ratio (FN/FP) = {cost_fn/cost_fp:.0f}:1")
print(f"\nOptimal threshold: {optimal_thresh:.4f}")
print(f"Total cost at optimal: ${optimal_cost:,.0f}")
print(f"\nConfusion matrix at optimal threshold:")
print(f"  TP={tp:,}, FP={fp:,}, TN={tn:,}, FN={fn:,}")
print(f"  Total cost = {fn}×${cost_fn:,} + {fp}×${cost_fp:,} = ${optimal_cost:,.0f}")

# Compare with default threshold (0.5)
y_pred_default = (y_pred_test_lr >= 0.5).astype(int)
cm_default = confusion_matrix(y_test, y_pred_default)
tn_d, fp_d, fn_d, tp_d = cm_default.ravel()
cost_default = fn_d * cost_fn + fp_d * cost_fp

print(f"\nComparison with default (0.5):")
print(f"  FN={fn_d}, FP={fp_d}")
print(f"  Total cost = ${cost_default:,.0f}")
print(f"  Savings = ${cost_default - optimal_cost:,.0f} ({(cost_default - optimal_cost)/cost_default*100:.1f}%)")

# Visualize cost vs threshold
fig, ax = plt.subplots(figsize=(10, 6))
thresholds_plot = np.linspace(0, 1, 1000)
ax.plot(thresholds_plot, all_costs, linewidth=2, color='steelblue')
ax.axvline(optimal_thresh, color='red', linewidth=2, linestyle='--', 
           label=f'Optimal={optimal_thresh:.3f}')
ax.axvline(0.5, color='orange', linewidth=2, linestyle='--', alpha=0.7,
           label='Default=0.5')
ax.set_xlabel('Threshold', fontsize=12)
ax.set_ylabel('Total Cost ($)', fontsize=12)
ax.set_title('Cost Minimization: Finding Optimal Threshold', fontsize=13, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(alpha=0.3)
plt.tight_layout()
plt.savefig(reports_dir / '03_cost_optimization.png', dpi=150, bbox_inches='tight')
plt.show()

**Solution 3**: When R² fails

**Answer**: Deploy Model A despite lower R².

**Reasoning**:

1. **R² limitations**:
   - Measures correlation, not prediction quality
   - Doesn't penalize impossible predictions (negative prices)
   - Can be high even with systematically biased predictions
   
2. **Model B red flags**:
   - **Negative price predictions are physically impossible** ❌
   - Indicates model doesn't respect domain constraints
   - May have extrapolation issues
   - Unreliable for production deployment
   
3. **Model A advantages**:
   - **Predictions always plausible** (within 20% of true) ✅
   - Lower R² but more robust
   - Predictable error bounds
   - Safe for production

**Additional metrics to check**:

```python
# 1. Mean Absolute Error (MAE)
#    → Interpretable: average $ error
mae = np.mean(np.abs(y_pred - y_true))

# 2. Mean Absolute Percentage Error (MAPE)
#    → Relative error (% of true value)
mape = np.mean(np.abs((y_pred - y_true) / y_true)) * 100

# 3. Max Error
#    → Worst-case prediction error
max_error = np.max(np.abs(y_pred - y_true))

# 4. Residual distribution
#    → Check for systematic bias, outliers
residuals = y_pred - y_true
plt.hist(residuals, bins=50)

# 5. Prediction bounds check
#    → Verify all predictions are plausible
assert (y_pred >= 0).all(), "Negative prices detected!"
assert (y_pred <= max_plausible_price).all(), "Implausibly high prices!"
```

**Decision rule for production**:
- R² as model selection metric (among valid models)
- Domain constraints as hard requirements
- MAE/MAPE for interpretability
- Max error for risk assessment

In [ ]:
# Solution 3 (code demonstration)
print("When R² Fails: Model Selection Example")
print("=" * 60)
print("\nScenario: House price prediction")
print("  Model A: R²=0.85, all predictions within 20% of true")
print("  Model B: R²=0.90, but sometimes predicts negative prices")
print("\nDecision: Deploy Model A ✅")
print("\nReasoning:")
print("  1. R² doesn't capture domain constraints")
print("  2. Negative prices violate physical reality")
print("  3. Model B unreliable for production")
print("  4. Model A has predictable errors")
print("\nLesson: High R² ≠ Good model")
print("        Always check:")
print("          • Domain constraints (price ≥ 0)")
print("          • Residual distribution")
print("          • Max error magnitude")
print("          • Prediction plausibility")

---

## ✅ Notebook Complete!

**What you learned**:
1. ✅ Establish baselines (majority class, logistic regression)
2. ✅ Choose metrics aligned with business goals
3. ✅ Understand metric limitations (accuracy, R²)
4. ✅ Select thresholds based on cost/benefit
5. ✅ Compute confidence intervals via bootstrap

**Outputs saved**:
- `reports/03_roc_curve_baseline.png`
- `reports/03_pr_curve_baseline.png`
- `reports/03_confusion_matrix.png`
- `reports/03_bootstrap_distributions.png`
- `reports/03_baseline_metrics.csv`
- `reports/03_cost_optimization.png`

**Key takeaways**:
- Baselines establish minimum performance
- Accuracy misleads with imbalanced data
- AUC-ROC (discrimination) vs. PR-AUC (minority class focus)
- Threshold selection requires domain knowledge
- Bootstrap CIs quantify uncertainty

**Next notebook**: `04_gbm_model_and_hyperparameter_search.ipynb` — Train a gradient boosting model and tune hyperparameters systematically.